In [ ]:
#!/home/ektov1-av_ca-sbrf-ru/bin/python35
import os
import sys
curruser = os.environ.get('USER')

# sys.path.insert(0, '/opt/workspace/{user}/system/support_library/'.format(user=curruser))
# sys.path.insert(0, '/opt/workspace/{user}/libs/'.format(user=curruser))
# sys.path.insert(0, '/opt/workspace/{user}/system/labdata/lib/'.format(user=curruser))


# sys.path.insert(0, './../src')
sys.path.insert(0, '/opt/workspace/{user}/notebooks/support_library/'.format(user=curruser))
sys.path.insert(0, '/opt/workspace/{user}/libs/python3.5/site-packages/'.format(user=curruser))
sys.path.insert(0, '/opt/workspace/{user}/notebooks/labdata/lib/'.format(user=curruser))

#import tendo.singleton
import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(filename='./__move_dbs__.log',level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(name)s %(message)s')
logger = logging.getLogger(__name__)


import joblib
import json
from joblib import Parallel, delayed

from time import sleep
from itertools import islice
from multiprocessing import Pool, Process, JoinableQueue
from multiprocessing.pool import ThreadPool
from functools import partial
import subprocess
from threading import Thread
import time
from datetime import datetime

from transliterate import translit

from spark_connector import SparkConnector
from sparkdb_loader import spark
from connector import OracleDB
import pyspark
from pyspark import SparkContext, SparkConf, HiveContext
from pyspark.sql.window import Window
from pyspark.sql.functions import *
import pyspark.sql.functions as f
from pyspark.sql.types import *
from pyspark.sql.dataframe import DataFrame

import re
import pandas as pd
import numpy as np
from tqdm._tqdm_notebook import tqdm_notebook
from pathlib import Path
import shutil
import loader as load

# sing = tendo.singleton.SingleInstance()

# os.chdir('/opt/workspace/ektov1-av_ca-sbrf-ru/notebooks/Clickstream_Analytics/AutoUpdate/')
# os.chdir('/opt/workspace/{}/notebooks/clickstream/AutoUpdate/'.format(curruser))


##-----------------------------------------
conn_schema = 'dl_t_team_ds_kb_sme' #'dl_t_team_ds_kb_sme' 'dl_t_team_speech_analytics'
new_schema = 'sbx_team_digitcamp'
ga_schema = 'sklod_external_google_analytics'
##-----------------------------------------


def show(self, n=10):
    return self.limit(n).toPandas()

def typed_udf(return_type):
    '''Make a UDF decorator with the given return type'''

    def _typed_udf_wrapper(func):
        return f.udf(func,return_type)

def essense(channel: str, prod_cd: str):
    message = "{}: {} retargeting".format(channel, prod_cd)
    return message

essense_udf = f.udf(essense, StringType())

pyspark.sql.dataframe.DataFrame.show = show

def print_and_log(message: str):
    print(message)
    logger.info(message)
    return None


In [2]:
print("### Starting spark context. Run!")

sp = spark(schema=conn_schema,
           dynamic_alloc=False,
           numofinstances=10, 
           numofcores=8,
           kerberos_auth=False
           )
hive = sp.sql
print(sp.sc.version)

### Starting spark context. Run!
2.4.0.cloudera2


### MA_PRODUCT_DICT

In [3]:
prod_dct = hive.sql("select * from {}.{} where CRM_PRODUCT_ID is not Null".format(new_schema,'MA_PRODUCT_DICT'))

In [ ]:
prod_dct = prod_dct.filter("(PRODUCT_CD_MMB is not Null or PRODUCT_CD_KSB is not Null) and (PRODUCT_SHORT_NM is not Null)")
prod_dct.show()

## MA_MMB_OFFER_NONTOP

In [7]:
nontop = hive.sql("select * from {}.{}".format(new_schema, 'MA_MMB_OFFER_NONTOP'))

In [ ]:
print("\n".join(["{} {}".format(col.upper(), _type) for col, _type in nontop.dtypes]))

In [7]:
maxdttm = nontop.select(f.max('START_DTTM')).collect()
max_load_dt_str = datetime.strftime(maxdttm[0]['max(START_DTTM)'], format='%Y-%m-%d %H:%M:%S')
mindttm = nontop.select(f.min('START_DTTM')).collect()
min_load_dt_str = datetime.strftime(mindttm[0]['min(START_DTTM)'], format='%Y-%m-%d %H:%M:%S')

max_load_dt_str, min_load_dt_str

('2020-09-16 00:00:00', '2020-09-16 00:00:00')

In [8]:
nontop.cache()
nontop = nontop.select([col.lower() for col in nontop.columns])
nontop = nontop.withColumn('HASHINN',f.sha2('INN',numBits=256))

In [9]:
nontop.count()

31539996

In [10]:
def getProdCDFromId(product_dict_nm):
    def get_product(product_id, product_dict_nm):
        if product_id is None:
            return None

        for _id in product_dict_nm:
            if product_id == _id:
                return product_dict_nm[_id]

        return None

    return f.udf(lambda x: get_product(x,product_dict_nm))

w2=Window.orderBy('COEF_PRODUCT_CD')

product_dict_sp = hive.sql("select * from {}.{}".format(new_schema, 'MA_PRODUCT_DICT'))
# product_dict_sp = product_dict_sp.select([col.lower() for col in product_dict_sp.columns])
product_dict_sp = product_dict_sp.filter("PRODUCT_CD_MMB is not Null").select('ID', 'CRM_PRODUCT_ID')
product_dict_pd = product_dict_sp.collect()
product_dict_nm = {row.ID: row.CRM_PRODUCT_ID for row in product_dict_pd}
nontop          = nontop.withColumn('CRM_PRODUCT_ID', getProdCDFromId(product_dict_nm)('PRODUCT_ID'))

nontop = nontop.orderBy(f.col('COEF_PRODUCT_CD').desc())\
               .withColumn('row_num', f.row_number().over(w2))

rowmax = nontop.select(f.max('row_num').alias('max')).collect()[0]['max']
nontop = nontop.select('INN', 'HASHINN', 'CRM_PRODUCT_ID', 'COEF_PRODUCT_CD', 
                       f.round(f.col('row_num')/rowmax,2).alias('SCORE'))

In [13]:
mean_nontop = nontop.groupBy('CRM_PRODUCT_ID').agg(f.mean('COEF_PRODUCT_CD').alias('MEAN_POTENTIAL'))
mean_nontop.show()

,CRM_PRODUCT_ID,MEAN_POTENTIAL
0,1-2GZPAXO5,2.656011e+03
1,1-115HHZC,4.844413e+08
2,1-1GVO,6.776397e+01
3,1-294EQR2Y,5.998274e+02
4,1-1EAUI3HB,4.995471e+04
5,1-1SEHRJGN,2.229084e+09
6,1-3BGP3MY,3.647171e+04
7,1-OVUJN21,1.330355e+00
8,1-1GW0,1.583316e+08
9,1-115ER0K,1.350127e+00


In [93]:
nontop.select('HASHINN', 'CRM_PRODUCT_ID', 'SCORE').filter("SCORE > 0.5").show()

,HASHINN,CRM_PRODUCT_ID,SCORE
0,0ccfb898f4704a1b58aa10d123e5e9839c731db855c32e...,1-1C1UMBRV,0.51
1,283bd72d4c27b96b34d2f3dd2ef229de8df0a07c6f53e7...,1-1C1UMBRV,0.51
2,32464a93f63a7f9847fa0348533c21ae36c6fa0b10fc4c...,1-1C1UMBRV,0.51
3,df6ce4a88c17a470e8ba4637c85ae9970d4d191a0e4ff4...,1-1C1UMBRV,0.51
4,caba8dcf56349ac4e84dcee23fe13125518b51e79be7d6...,1-1C1UMBRV,0.51
5,69a44f7717b710c880ec8b787e53ff9c827d68c66541a8...,1-1C1UMBRV,0.51
6,b27142786057d28fda09f9b9f913577ad68b00e38f2892...,1-1C1UMBRV,0.51
7,3ad3456e65dd8ed44d5689c66ee083c41c28244875238a...,1-1C1UMBRV,0.51
8,641ec8029f1033f93f7cf66cd00c86edb090f1e4cbd302...,1-1C1UMBRV,0.51
9,6ccd78a23efb7c95bb8300b693e6ab282fba8bb45ae4b6...,1-1C1UMBRV,0.51


In [36]:
nontop =\
nontop.select('client_id',
                     f.col('crm_product_id').alias('i'),
                     f.split(f.col('sources_lv2_list'),',').getItem(0).alias('t'),
                     f.abs(f.log(np.e,f.col('offer_priority'))).alias('r'))
rowmax = nontop.select(f.max('r').alias('max')).collect()[0]['max']
nontop = nontop.select('client_id', 'i','t',f.round(f.col('r')/rowmax,2).alias('r'))
nontop = nontop.fillna({'t':''})

In [ ]:
sql = """
(
select /*+ parallel (8) */ count(*) from {}
)""".format('OFFER_PRIORITY')
res = sp.get_oracle(OracleDB('iskra4'), sql)

In [ ]:
res.show()

## OFFER_PRIORITY 

In [20]:
priority = hive.sql("select * from {}.{}".format(new_schema, 'OFFER_PRIORITY'))

In [ ]:
priority.dtypes

In [10]:
maxdttm = priority.select(f.max('LOAD_DTTM')).collect()
max_load_dt_str = datetime.strftime(maxdttm[0]['max(LOAD_DTTM)'], format='%Y-%m-%d %H:%M:%S')
mindttm = priority.select(f.min('LOAD_DTTM')).collect()
min_load_dt_str = datetime.strftime(mindttm[0]['min(LOAD_DTTM)'], format='%Y-%m-%d %H:%M:%S')

max_load_dt_str, min_load_dt_str

('2020-09-21 00:00:00', '2019-06-18 21:06:52')

In [21]:
priority_part = priority.filter("LOAD_DTTM >= timestamp('{}')".format('2020-06-01 00:00:00'))

In [22]:
priority_part.cache()
priority_part = priority_part.select([col.lower() for col in priority_part.columns])
priority_part = priority_part.withColumn('HASHINN',f.sha2('inn',numBits=256))
priority_part.count()

4539125

In [23]:
# rowmax = np.log(priority_part.select(f.max('offer_priority').alias('max')).collect()[0]['max'])
# priority_part = priority_part.withColumn('SCORE',
#                     f.when(f.col('offer_priority')>1, f.round(f.abs(f.log(np.e,f.col('offer_priority')))/rowmax,4).alias('SCORE'))\
#                      .otherwise(f.round(f.col('offer_priority'),4).alias('SCORE'))
#                                         )

w2=Window.orderBy('offer_priority')

priority_part = priority_part.orderBy(f.col('offer_priority').desc())\
                             .withColumn('row_num', f.row_number().over(w2))

rowmax = priority_part.select(f.max('row_num').alias('max')).collect()[0]['max']
priority_part = priority_part.select('INN', 'HASHINN', 'CRM_PRODUCT_ID', 'offer_priority',
                                     f.round(f.col('row_num')/rowmax,2).alias('SCORE'))

In [24]:
mean_priority = priority_part.groupBy('CRM_PRODUCT_ID').agg(f.mean('offer_priority').alias('MEAN_POTENTIAL'))
mean_priority.show()

,CRM_PRODUCT_ID,MEAN_POTENTIAL
0,1-115HHZC,2.961480e+08
1,1-1GVO,2.209147e+02
2,1-294EQR2Y,1.398958e+03
3,1-1EAUI3HB,5.622541e+04
4,1-1SEHRJGN,2.482302e+08
5,1-3BGP3MY,1.005323e+07
6,1-WCK2VTA,1.450146e+01
7,1-1GW0,6.095668e+08
8,1-DDP18DU,2.756600e+03
9,1-79LMUVD,3.483464e+09


In [99]:
priority_part.filter("SCORE > 0.5").show()

,HASHINN,CRM_PRODUCT_ID,SCORE
0,14a98d356cd8ddb556e84ae64fb496b9d4125bda6a2fdd...,1-1GW0,0.51
1,c03497ab6c4d4102504fc1d9298ddde2bbff8bf3ea626e...,1-1GW0,0.51
2,107ca3d181d17da843c0ef0d0661d187debbb415a6e0a6...,1-1GW0,0.51
3,184e25653ca31b23fa8c961d83ca9506774598c3e11c46...,1-1GW0,0.51
4,a6009cc6e08e58d62038ce21c1830cf96c742402076d65...,1-1GW0,0.51
5,a0b9cce47df89c46e0eb16d6b050bc16747cb9ee3ce573...,1-1GW0,0.51
6,16b4367725722302ac0ded25ca7c0ca17d0845b5a8663e...,1-1GW0,0.51
7,2edb9608d8eb1ea601b7d6f67fabfa6000fcb7e6731c2b...,1-1GW0,0.51
8,f7f447d928c1b52a925a17569df0ae8eec17a821d4a47f...,1-1GW0,0.51
9,7526cb6460e380604867d5f725dcde2b56ffb605c9d27c...,1-1GW0,0.51


In [41]:
priority_part =\
priority_part.select('client_id',
                     f.col('crm_id').alias('i'),
                     f.split(f.col('sources_lv2_list'),',').getItem(0).alias('t'),
                     f.abs(f.log(np.e,f.col('offer_priority'))).alias('r'))
rowmax = priority_part.select(f.max('r').alias('max')).collect()[0]['max']
priority_part = priority_part.select('client_id','i','t',f.round(f.col('r')/rowmax,2).alias('r'))
priority_part = priority_part.fillna({'t':''})

### UNION MEAN_POTENTIALS

In [26]:
mean_total = mean_nontop.unionAll(mean_priority).distinct()
mean_total.cache()
mean_total.count()

131

### JOIN with MA_PRODUCT_DICT

In [40]:
df = prod_dct.join(mean_total, on='CRM_PRODUCT_ID', how='left_outer').toPandas()
df.shape

(169, 12)

In [ ]:
df = df[~df['MEAN_POTENTIAL'].isnull()]
df.sort_values(by=['MEAN_POTENTIAL'], ascending=False, inplace=True)
df.head(20)

In [39]:
df.shape

(130, 12)

In [42]:
df.to_excel("./ma_prod_dict.xlsx", index=False)

### GA_ALL_SCENARIOS_HIST

In [11]:
ga = hive.table("{}.{}".format(new_schema, 'GA_ALL_SCENARIOS_HIST'))

In [12]:
ga = ga.filter("(load_dt > timestamp('2020-07-01')) and \
                (sum_offer_priority > 0 or sum_offer_nontop > 0)").select('inn', 'id_product', 'funnel_rate').distinct()

In [13]:
def getProdCDFromId(product_dict_nm):
    def get_product(product_id, product_dict_nm):
        if product_id is None:
            return None

        for _id in product_dict_nm:
            if product_id == _id:
                return product_dict_nm[_id]

        return None

    return f.udf(lambda x: get_product(x,product_dict_nm))


ga.cache()
ga = ga.withColumn('HASHINN',f.sha2('inn',numBits=256))
ga = ga.withColumn('SCORE', f.when(f.col('funnel_rate')==9999, 1).otherwise(f.col('funnel_rate')))

product_dict_sp = hive.sql("select * from {}.{}".format(new_schema, 'MA_PRODUCT_DICT'))
# product_dict_sp = product_dict_sp.select([col.lower() for col in product_dict_sp.columns])
product_dict_sp = product_dict_sp.filter("PRODUCT_CD_MMB is not Null").select(f.col('PRODUCT_CD_MMB').alias('ID'), 
                                                                              'CRM_PRODUCT_ID')
product_dict_pd = product_dict_sp.collect()
product_dict_nm = {row.ID: row.CRM_PRODUCT_ID for row in product_dict_pd}
ga              = ga.withColumn('CRM_PRODUCT_ID', getProdCDFromId(product_dict_nm)('id_product'))



ga = ga.select('HASHINN', 'SCORE', 'CRM_PRODUCT_ID')
ga.count()

399988

In [117]:
ga.show()

,HASHINN,SCORE,CRM_PRODUCT_ID
0,a951376451b08966251d6204e0ab3f06e1c550555ed553...,0.50,1-115HHZO
1,c3ef33c105d673e06080ef89e3aedd88ab16f18ede2584...,1,1-1GUC
2,31334905ca7585baccd7723469fbcc9aea405efba404f0...,1.0,1-1GW0
3,6ad50f8ac9884cd7ff0acbaf60f73cfdec5db5a19a0610...,0.60,1-1GVC
4,d4937d76a5d08a9d91542a8e14b00e8307ef54124ec0b2...,0.80,1-1GVC
5,8a60b5f4a7ec23174de65f9a76f8fc38ddae301c226376...,0.33,1-1GT0
6,067b955fe6010ecf13c363fb69864b812a781cf9171125...,1,1-115HHZO
7,3de08b1450399001a88f80c1c09f0004f7b2ad50746239...,1.00,1-1GT0
8,fc66ccc954300b26e448e16a140a4b96efe107510e9e3b...,0.33,1-1GT0
9,9ddf8d876837227f828a8db3003e1840bd4d40f073d1b1...,0.80,1-1GVC


## Union dbs

In [14]:
all_sdf = priority_part.unionAll(nontop).unionAll(ga).distinct()
all_sdf = all_sdf.filter("HASHINN is not Null")
all_sdf.cache()

DataFrame[HASHINN: string, CRM_PRODUCT_ID: string, SCORE: string]

In [15]:
all_sdf.count()

35458452

In [36]:
all_sdf = all_sdf.withColumn('PRODUCT_TYPE', f.lit('STANDART'))\
                 .withColumn('PRODUCT_CATEGORY', f.lit('BANK'))
all_sdf.show()

,HASHINN,CRM_PRODUCT_ID,SCORE,PRODUCT_TYPE,PRODUCT_CATEGORY
0,51234ad24a879819f9d529202f8560b4e3a67c94f8bce7...,1-28U2XVNP,0.0,STANDART,BANK
1,9a0cffd90ac73a0997c35804a906595691cfb7e2e72f9a...,1-28U2XVNP,0.0,STANDART,BANK
2,9decd2f3cb790ba1b492dcd98e1ec510c012f2031658fc...,1-28U2XVNP,0.0,STANDART,BANK
3,6fb512136c40c617c69552073bd8efb21e7649ea061f77...,1-28U2XVNP,0.0,STANDART,BANK
4,4b6f9eaba7b20c035a7af14c20a40c7036b76968a818cc...,1-28U2XVNP,0.0,STANDART,BANK
5,83d784192fd5560adfef7099987b3046cded08784faae5...,1-28U2XVNP,0.0,STANDART,BANK
6,d106330ac63263d686250989f06318bfe437859a68aa5b...,1-28U2XVNP,0.0,STANDART,BANK
7,76e6b750e2cb73646998204c5dcd738691576a6030a427...,1-28U2XVNP,0.0,STANDART,BANK
8,24dcf7766c6971f85da233e010b75c2204c93adaaa879f...,1-28U2XVNP,0.0,STANDART,BANK
9,07a760137254fb6430a36a579dfe00a9208bb5945b22e1...,1-28U2XVNP,0.0,STANDART,BANK


In [18]:
all_sdf.select("HASHINN").take(3)

[Row(HASHINN='51234ad24a879819f9d529202f8560b4e3a67c94f8bce74cd9b468ada06dee23'),
 Row(HASHINN='9a0cffd90ac73a0997c35804a906595691cfb7e2e72f9ae7fd55da6ec557336d'),
 Row(HASHINN='9decd2f3cb790ba1b492dcd98e1ec510c012f2031658fcba6db070dfc7febb64')]

In [39]:
res = all_sdf\
         .withColumn('ROW_NUM', f.row_number().over(Window.partitionBy('HASHINN', 'CRM_PRODUCT_ID')\
                                              .orderBy(f.col('SCORE').desc())))\
         .filter("(ROW_NUM = 1) and (SCORE > 0.1)")\
         .drop('ROW_NUM')

In [40]:
res.orderBy(f.col("HASHINN"), f.col('SCORE').desc())\
   .filter("HASHINN='9a0cffd90ac73a0997c35804a906595691cfb7e2e72f9ae7fd55da6ec557336d'").show(30)

,HASHINN,CRM_PRODUCT_ID,SCORE,PRODUCT_TYPE,PRODUCT_CATEGORY
0,9a0cffd90ac73a0997c35804a906595691cfb7e2e72f9a...,1-115HHZC,0.97,STANDART,BANK
1,9a0cffd90ac73a0997c35804a906595691cfb7e2e72f9a...,1-1GVC,0.92,STANDART,BANK
2,9a0cffd90ac73a0997c35804a906595691cfb7e2e72f9a...,1-PJ1ZWPY,0.88,STANDART,BANK
3,9a0cffd90ac73a0997c35804a906595691cfb7e2e72f9a...,1-1GW0,0.86,STANDART,BANK
4,9a0cffd90ac73a0997c35804a906595691cfb7e2e72f9a...,1-1Z85A72,0.79,STANDART,BANK
5,9a0cffd90ac73a0997c35804a906595691cfb7e2e72f9a...,1-ZYQP07F,0.65,STANDART,BANK
6,9a0cffd90ac73a0997c35804a906595691cfb7e2e72f9a...,1-MP71XI8,0.59,STANDART,BANK
7,9a0cffd90ac73a0997c35804a906595691cfb7e2e72f9a...,1-JNGJROI,0.46,STANDART,BANK
8,9a0cffd90ac73a0997c35804a906595691cfb7e2e72f9a...,1-1BTGN115,0.3,STANDART,BANK
9,9a0cffd90ac73a0997c35804a906595691cfb7e2e72f9a...,1-ZYQO0PN,0.21,STANDART,BANK


In [41]:
res.count()

29012974

In [45]:
part = res.sample(False, fraction=0.015).toPandas()

In [48]:
part.shape

(434288, 5)

### SALE_POTENTIAL

In [50]:
from csv import  QUOTE_MINIMAL, QUOTE_ALL

part.to_csv('./sale_potential.csv', 
            index=False, 
            encoding='utf-8', 
            float_format='%.2f',
            sep=',', 
            doublequote=True,
            quotechar='\"',
            quoting=QUOTE_MINIMAL)

### UNIFIED_CUSTOMER

In [10]:
cust = hive.table("{}.{}".format(new_schema, 'UNIFIED_CUSTOMER'))

In [80]:
cust = cust.withColumn('HASHINN',f.sha2('INN',numBits=256))\
           .withColumn('CLIENT_FLG', f.lit(1))   

In [81]:
cust = cust.select("HASHINN", 
                   f.col("CRM_SUB_INDUSTRY_TYPE_NM").alias('INDUSTRY'), 
                   'CLIENT_FLG').filter("INDUSTRY is not Null")

In [82]:
unified_cust = cust.limit(1600000).toPandas()

In [83]:
unified_cust.head()

,HASHINN,INDUSTRY,CLIENT_FLG
0,0e027e9a8b25746ccd57af2ed93fef41a4422268ea6127...,Офисная недвижимость,1
1,399543a893cd0d82e702bc1cd322403349401b4d0d24e1...,Торговля продовольственными товарами,1
2,f07986595a8cb733ee21ddad59bc8a578ff49f8d48062d...,Торговля продовольственными товарами,1
3,3d93b3f4cb8590c12829446436ad438591b773abd4e7ce...,"Программное обеспечение, информационные технол...",1
4,42af0c6bcc640bbf3d96902060dc59a4502eaddab5aa18...,Перевозка автотранспортом,1


In [146]:
from csv import  QUOTE_MINIMAL, QUOTE_ALL

unified_cust.to_csv('./inn_attributes.csv', 
            index=False, 
            encoding='utf-8', 
            float_format='%.2f',
            sep=',', 
            doublequote=True,
            quotechar='\"',
            quoting=QUOTE_MINIMAL)

## CID GA

In [54]:
currdate = datetime.strptime(datetime.strftime(datetime.today(), format='%Y-%m-%d'), '%Y-%m-%d')
actualts = datetime.timestamp(currdate) - 10 * 30 * 24 * 60 * 60
max_resp_dt_str = datetime.strftime(datetime.fromtimestamp(actualts),  format='%Y-%m-%d')
max_resp_dt_str

'2020-06-06'

In [55]:
custcid = hive.table("{}.{}".format(new_schema, 'GA_CID_SBBOL_INN')).filter(f.col("load_dt") >= pd.to_datetime(max_resp_dt_str))

In [56]:
custcid = custcid.select("cid", "cu_inn").filter("cu_inn is not Null")

In [57]:
cookie = custcid.distinct()

In [58]:
cookie.cache()

DataFrame[cid: string, cu_inn: string]

In [59]:
cookie.count()

19525339

In [60]:
cookie = cookie.withColumn('HASHINN', f.sha2('cu_inn',numBits=256))
cookie = \
cookie.select('HASHINN',
              f.col('cid').alias('COOKIE_ID'),
              f.lit('GA').alias('COOKIE_VENDOR'))

In [25]:
cookie.show()

,HASHINN,COOKIE_ID,COOKIE_VENDOR
0,7af4f7778bfb40d810862299523356006501f7810948e2...,1686232064.1426579796,GA
1,da15b85aed2cdd6efe0de7f460e76186d7325682314b9a...,635473949.1614241308,GA
2,8cb990e35a6455feda4214610fff7159cb996d5bad7ebc...,885540059.1598337530,GA
3,0f01c07af73dd2069d24a3b1b0dc71450f9fa2915635d6...,526876878.1554105223,GA
4,0fbddcdeca3390df9fb798d4b4a556a1d7e7e6c2070856...,1690983206.1610627846,GA
5,f1101e469cc6c2c081473ff33e3efb84a33a4b17822e45...,1590633656.1554386362,GA
6,3a851788faaf7e0471af494e20a5655b416bfd4b8a947c...,1258673748.1599545758,GA
7,6c86a55ea8d1110208fb53d69d9dbeb6a3e7b9fca89a74...,1172678957.1584253949,GA
8,1716ace82a06415b28daf400e852b04ecd63eb419ee24f...,885901514.1523010331,GA
9,35e5a342f8a525ed4f22778282fa5abdf01727aa796b5a...,53762997.1601382396,GA


In [50]:
hashinn = pd.read_csv("./csv/TESTHASH.csv")
hashinn.count()

HASHINN    6166
dtype: int64

In [61]:
hashinn_sdf = hive.createDataFrame(hashinn,schema=StructType([StructField("HASHINN_",StringType())]))

In [62]:
conditions = (cookie.HASHINN == hashinn_sdf.HASHINN_)
res = cookie.join(hashinn_sdf, how="inner", on=conditions)

In [64]:
res = res.select("HASHINN","COOKIE_ID","COOKIE_VENDOR")
res = res.distinct()
res.count()

5995

In [66]:
part = res.toPandas()

In [67]:
part.to_csv("./ga_cookies", index=False)

### MA_CUSTOMER_COOKIE_MAP

In [3]:
ISKRA_TABLE = 'MA_CUSTOMER_COOKIE_MAP'

In [4]:
currdate = datetime.strptime(datetime.strftime(datetime.today(), format='%Y-%m-%d'), '%Y-%m-%d')
actualts = datetime.timestamp(currdate) - 40 * 24 * 60 * 60
max_resp_dt_str = datetime.strftime(datetime.fromtimestamp(actualts),  format='%Y-%m-%d %H:%M:%S')
max_resp_dt_str

'2020-08-20 00:00:00'

In [5]:
db = OracleDB('iskra4')

sql = """
(
select /*+ parallel (8) */ * from {}
where SRC_UPDATE_DTTM > to_timestamp('{}', 'yyyy-mm-dd hh24:mi:ss')
)""".format(ISKRA_TABLE, max_resp_dt_str)

cookie = sp.get_oracle(db, sql)

In [6]:
cookie = cookie.withColumn('HASHINN', f.sha2('CUST_INN',numBits=256))

In [7]:
cookie = \
cookie.select('HASHINN',
              f.col('CUST_COOKIE').alias('COOKIE_ID'),
              f.col('SRC_SYSTEM_CD').alias('COOKE_VENDOR'))

In [8]:
coolie_df = cookie.limit(1000000).toPandas()

In [9]:
from csv import  QUOTE_MINIMAL, QUOTE_ALL

coolie_df.to_csv('./inn_cookies.csv', 
            index=False, 
            encoding='utf-8', 
            float_format='%.2f',
            sep=',', 
            doublequote=True,
            quotechar='\"',
            quoting=QUOTE_MINIMAL)

## Get Json

In [27]:
from pyspark.sql import Row
import json

In [47]:
res = all_sdf.rdd.map(lambda row: ((row.asDict()['client_id']),
                                   {k:v for k,v in row.asDict().items() if k!='client_id'})).groupByKey().map(lambda x: (x[0], json.dumps(list(x[1]), 
                                                                                                                                          ensure_ascii=False,
                                                                                                                                          separators=(',', ':'))))

In [48]:
sdf = res.toDF(schema=StructType([StructField('client_id', StringType()),
                                  StructField('products',  StringType())])) #columns=['client_id', 'products'])

In [50]:
sdf.take(1)

[Row(client_id='946b6b3322252e639bfd69925dbb245f58138d855505925b7d367cfbeb9875ae', products='[{"i":"1-CZALTM","r":0.17,"t":"OTR2"},{"i":"1-CZALTM","r":0.06,"t":"ЦА-Т039ММБ-0120"},{"i":"1-CZALTM","r":0.26,"t":"ЦА-Т039ММБ-0120"},{"i":"1-CZALTM","r":0.07,"t":"ЦА-Т039ММБ-0120"},{"i":"1-CZALTM","r":0.04,"t":"OTR2"}]')]

In [51]:
sdf = sdf.select([col.upper() for col in sdf.columns])

##  Export via JDBC driver

In [2]:
ld=load.Loader(init_dsn=True, encoding='cp1251',  sep=',')
db = load.OracleDB('', '', ld._get_dsn('')) 
db.connect()

In [53]:
table_name = 'mmb_json_prods'

sql = "DROP TABLE {}".format(table_name)
db.cursor.execute(sql)
db.connection.commit()
print('dropping done')

dropping done


In [55]:
table_name = 'productoffer_mmb_json'

typesmap={}
for column_name, column in sdf.dtypes:
    if column == 'string':
        if 'INN' in column_name.upper() or 'KPP' in column_name:
            typesmap[column_name] = 'VARCHAR(20)'
        elif 'PRODUCTS'.upper() in column_name:
            typesmap[column_name] = 'VARCHAR(4000)'
        else:
            typesmap[column_name] = 'VARCHAR(200)'
    elif column == 'int':
        typesmap[column_name] = 'INTEGER'
    elif column == 'bigint':
        typesmap[column_name] = 'INTEGER'
    elif column == 'timestamp':
        typesmap[column_name] = 'TIMESTAMP'
    elif column == 'float' or column == 'double':
        typesmap[column_name] = 'FLOAT'
    else:
        None

cols = ', '.join([col + ' ' + typesmap[col] for col in sdf.columns])

db = OracleDB('iskra4')
mode = 'append'
sdf \
    .write \
    .format('jdbc') \
    .mode(mode) \
    .option('url', 'jdbc:oracle:thin:@//'+db.dsn) \
    .option('user', db.user) \
    .option('password', db.password) \
    .option('dbtable', table_name) \
    .option('createTableColumnTypes', cols)\
    .option('driver', 'oracle.jdbc.driver.OracleDriver') \
    .save()

## Check saved table by fetching its rows via cx_Oracle driver

In [86]:
ld=load.Loader(init_dsn=True, encoding='cp1251',  sep=',')
db = load.OracleDB('', '', ld._get_dsn('')) 
db.connect()

In [9]:
ISKRA_TABLE = 'productoffer_mmb_json'

query = '''
select /*+ parallel (8) */
          *
FROM {}
where client_id is not Null
'''.format(ISKRA_TABLE)
df = pd.read_sql(query, con=db.connection)

In [7]:
df.head(15)

,CLIENT_ID,PRODUCTS
0,9d17d514ec5d528789229fca75519e2c3b52b8dd0cdbe3...,"[{""i"":""1-2COMTKH"",""r"":0.4,""t"":""MRK""},{""i"":""1-2..."
1,7a389c0c806ef61bc63929410ee9dd4af7539d1b9f81e2...,"[{""i"":""1-1YIGDFL"",""r"":0.05,""t"":""OTR2""},{""i"":""1..."
2,dc6bd4935b259d16421bafe112b67dc6b3d76abeb81cf8...,"[{""i"":""1-JZPYP4D"",""r"":0.06,""t"":""OTR2""},{""i"":""1..."
3,236f10eed1dce4452a9dd9c68a19bd44bba04d28c4114f...,"[{""i"":""1-TWCOITI"",""r"":0.09,""t"":""OTR2""}]"
4,cebc092e45dba10869e8b8c19076995ae57eb632a09c20...,"[{""i"":""1-PV5Q99N"",""r"":0.09,""t"":""ЦА-КРЕДСМАРТ-0..."
5,b963e304f4660b6e869eda153683bd53d6885eb6a46195...,"[{""i"":""1-287PXE6B"",""r"":0.05,""t"":""OTR2""},{""i"":""..."
6,3b0a860e7e8e9d3333c5113a6d33d084b6b32efdadf365...,"[{""i"":""1-ZKLZZNC"",""r"":0.07,""t"":""OTR2""},{""i"":""1..."
7,3938adff80d20aef6e3c42f4d468c59f420942eed30b73...,"[{""i"":""1-55F5PH7"",""r"":0.09,""t"":""OTR2""},{""i"":""1..."
8,42e7d51affa7d0efcd290552c70d563dbd765bbda4407e...,"[{""i"":""1-2SSEF6H"",""r"":0.09,""t"":""MRK""},{""i"":""1-..."
9,222bcd957ddea4021a37a9bee81bc7f87444fafab13507...,"[{""i"":""1-OBHF29U"",""r"":0.23,""t"":""OTR2""},{""i"":""1..."


In [11]:
db.close()

## Sampling a part from Dbs

In [ ]:
sdf = sp.get_oracle(db, table_name)

In [56]:
part = sdf.limit(15000).toPandas()

In [59]:
from csv import  QUOTE_MINIMAL, QUOTE_ALL

In [60]:
part.to_csv('./productoffer_mmb_json.csv', 
            index=False, 
            encoding='utf-8', 
            float_format='%.2f',
            sep=',', 
            doublequote=True,
            quotechar='\"',
            quoting=QUOTE_MINIMAL)

## Export to csv in HDFS

In [ ]:
path_to_hdfs=os.path.join('/user', os.environ.get("USER"), 'csv')
sdf.coalesce(5).write \
                .format('com.databricks.spark.csv') \
                .mode('overwrite') \
                .option('sep', ',') \
                .option('header', True) \
                .option("quote", '"') \
                .save(path_to_hdfs)

In [ ]:
_sdf = hive.read.format("com.databricks.spark.csv") \
                .option('header', 'true') \
                .option('delimiter', ',') \
                .option('decimal', '.') \
                .option('encoding', 'utf-8') \
                .load(path_to_hdfs)